In [1]:
import itertools
import random
import numpy as np
import sys, os
import pandas as pd 
import torch
from torchsummary import summary
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset,DataLoader,RandomSampler
import torch.nn.functional as F
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
# from apex import amp
import shutil
import tensorflow as tf


# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

SEED = 7219
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/backend.py:5747: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/.keras/keras.json' mode='r' encoding='ANSI_X3.4-1968'>
  _config = json.load(open(_config_path))


In [2]:
# Set the device and empty cache 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# cudnn.benchmark = True
torch.cuda.empty_cache()
device

device(type='cuda')

In [3]:
class CreateDataset(Dataset):
    def __init__(self,data,atten_mask,labels):
        self._dataset = [[data[i],atten_mask[i],labels.values[i]] for i in range(0,len(data))]
    
    def __len__(self):
        return len(self._dataset)

    def __getitem__(self,idx):
        return self._dataset[idx]

In [4]:
# read back tokenized data
TOKEN_DATA_PATH = '/root/data/BERT/uncased_L-12_H-768_A-12-pytorch/tokenized/'
with open(TOKEN_DATA_PATH+'train_data_tokenized.txt', 'rb') as fp:
    train_data_tokenized=pickle.load(fp)
with open(TOKEN_DATA_PATH+'train_attention_mask.txt', 'rb') as fp:
    train_attention_mask=pickle.load(fp)    
with open(TOKEN_DATA_PATH+'valid_data_tokenized.txt', 'rb') as fp:
    valid_data_tokenized=pickle.load(fp)
with open(TOKEN_DATA_PATH+'valid_attention_mask.txt', 'rb') as fp:
    valid_attention_mask=pickle.load(fp)
with open(TOKEN_DATA_PATH+'test_data_tokenized.txt', 'rb') as fp:
    test_data_tokenized=pickle.load(fp)
with open(TOKEN_DATA_PATH+'test_attention_mask.txt', 'rb') as fp:
    test_attention_mask=pickle.load(fp)
with open(TOKEN_DATA_PATH+'train_labels.txt', 'rb') as fp:
    train_labels=pickle.load(fp)
with open(TOKEN_DATA_PATH+'valid_labels.txt', 'rb') as fp:
    valid_labels=pickle.load(fp)
with open(TOKEN_DATA_PATH+'train_labels.txt', 'rb') as fp:
    test_labels=pickle.load(fp)

In [5]:
train_dataset = CreateDataset(train_data_tokenized,train_attention_mask,train_labels)
train_sampler = RandomSampler(train_dataset)
valid_dataset = CreateDataset(valid_data_tokenized,valid_attention_mask,valid_labels)
valid_sampler = RandomSampler(valid_dataset)

In [6]:
dummy_input = valid_dataset[0]
dum_input = torch.unsqueeze(dummy_input[0],0)
dum_attn_mask = torch.unsqueeze(valid_attention_mask[0],0)
dum_labels = dummy_input[1][0]

In [7]:
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [ ]:
tf_graph = load_pb('./model_simple.pb')
sess = tf.Session(graph=tf_graph)

# Show tensor names in graph
for op in tf_graph.get_operations():
    print(op.values())

output_tensor = tf_graph.get_tensor_by_name('test_output:0')
input_tensor = tf_graph.get_tensor_by_name('test_input:0')


output = sess.run(output_tensor, feed_dict={input_tensor: dum_input})
# print(output)

(<tf.Tensor 'Const:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_1:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_2:0' shape=(512, 768) dtype=float32>,)
(<tf.Tensor 'Const_3:0' shape=(2, 768) dtype=float32>,)
(<tf.Tensor 'Const_4:0' shape=(30522, 768) dtype=float32>,)
(<tf.Tensor 'Const_5:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_6:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_7:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_8:0' shape=(768, 768) dtype=float32>,)
(<tf.Tensor 'Const_9:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_10:0' shape=(768, 768) dtype=float32>,)
(<tf.Tensor 'Const_11:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_12:0' shape=(768, 768) dtype=float32>,)
(<tf.Tensor 'Const_13:0' shape=(768,) dtype=float32>,)
(<tf.Tensor 'Const_14:0' shape=(768, 768) dtype=float32>,)
(<tf.Tensor 'Const_15:0' shape=(3072,) dtype=float32>,)
(<tf.Tensor 'Const_16:0' shape=(3072, 768) dtype=float32>,)
(<tf.Tensor 'Const_17:0' shape=(768,) dtype

(<tf.Tensor 'Add_6:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Shape_4:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_209:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_7/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_7:0' shape=() dtype=int64>,)
(<tf.Tensor 'Shape_5:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_210:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_8/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_8:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_211:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_212:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_10/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_10:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_11/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_11:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_12/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_12:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_13/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_13:

(<tf.Tensor 'Const_243:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_244:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_29/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_29:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_30/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_30:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_31/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_31:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_32/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_32:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'concat_8/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'concat_8:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Shape_26:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_245:0' shape=() dtype=int64>,)
(<tf.Tensor 'Equal_5:0' shape=(4,) dtype=bool>,)
(<tf.Tensor 'Where_6:0' shape=(?, 1) dtype=int64>,)
(<tf.Tensor 'Squeeze_6:0' shape=(?,) dtype=int64>,)
(<tf.Tensor 'GatherV2_24/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_24:0' 

(<tf.Tensor 'GatherV2_40:0' shape=() dtype=int64>,)
(<tf.Tensor 'Shape_49:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Const_280:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_41/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_41:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_281:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_52/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_52:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_53/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_53:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_54/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_54:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'concat_14/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'concat_14:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Shape_50:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Const_282:0' shape=() dtype=int64>,)
(<tf.Tensor 'Equal_11:0' shape=(3,) dtype=bool>,)
(<tf.Tensor 'Where_12:0' shape=(?, 1) dtype=int64>,)
(<tf.Tensor 'Squeeze_12:0' shape=(?

(<tf.Tensor 'Add_71:0' shape=(1, 128, 3072) dtype=float32>,)
(<tf.Tensor 'Mul_16:0' shape=(1, 128, 3072) dtype=float32>,)
(<tf.Tensor 'Const_314:0' shape=() dtype=float32>,)
(<tf.Tensor 'Mul_17:0' shape=(1, 128, 3072) dtype=float32>,)
(<tf.Tensor 'transpose_41/perm:0' shape=(2,) dtype=int32>,)
(<tf.Tensor 'transpose_41:0' shape=(3072, 768) dtype=float32>,)
(<tf.Tensor 'MatMul_31:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Add_72:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Add_73:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Mean_16/reduction_indices:0' shape=() dtype=int32>,)
(<tf.Tensor 'Mean_16:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'Sub_9:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Const_315:0' shape=() dtype=float32>,)
(<tf.Tensor 'Pow_8:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Mean_17/reduction_indices:0' shape=() dtype=int32>,)
(<tf.Tensor 'Mean_17:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'Const_316:0' shape=() dty

(<tf.Tensor 'GatherV2_68/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_68:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_346:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_347:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_85/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_85:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_86/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_86:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_87/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_87:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_88/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_88:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'concat_23/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'concat_23:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Shape_86:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_348:0' shape=() dtype=int64>,)
(<tf.Tensor 'Equal_20:0' shape=(4,) dtype=bool>,)
(<tf.Tensor 'Where_21:0' shape=(?, 1) dtype=int64>,)
(<tf.Tensor 'Squeeze_21:

(<tf.Tensor 'GatherV2_88/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_88:0' shape=() dtype=int64>,)
(<tf.Tensor 'Shape_113:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Const_388:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_89/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_89:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_389:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_112/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_112:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_113/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_113:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_114/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_114:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'concat_30/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'concat_30:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Shape_114:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Const_390:0' shape=() dtype=int64>,)
(<tf.Tensor 'Equal_27:0' shape=(3,) dtype=bool>,)
(<tf.Tensor 'Where_28:0

(<tf.Tensor 'Reshape_32:0' shape=(1, 128, 12, 64) dtype=float32>,)
(<tf.Tensor 'transpose_85/perm:0' shape=(4,) dtype=int32>,)
(<tf.Tensor 'transpose_85:0' shape=(1, 12, 128, 64) dtype=float32>,)
(<tf.Tensor 'Shape_136:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_430:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_106/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_106:0' shape=() dtype=int64>,)
(<tf.Tensor 'Shape_137:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_431:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_107/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_107:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_432:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_433:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_134/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_134:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_135/dim:0' shape=() dtype=int32>,)
(<tf.Tensor 'ExpandDims_135:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'ExpandDims_136/dim:0' shape=() dt

(<tf.Tensor 'Mean_38:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'Sub_20:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Const_472:0' shape=() dtype=float32>,)
(<tf.Tensor 'Pow_19:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Mean_39/reduction_indices:0' shape=() dtype=int32>,)
(<tf.Tensor 'Mean_39:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'Const_473:0' shape=() dtype=float32>,)
(<tf.Tensor 'Add_176:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'Sqrt_19:0' shape=(1, 128, 1) dtype=float32>,)
(<tf.Tensor 'div_38:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Mul_39:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'Add_177:0' shape=(1, 128, 768) dtype=float32>,)
(<tf.Tensor 'transpose_100/perm:0' shape=(2,) dtype=int32>,)
(<tf.Tensor 'transpose_100:0' shape=(768, 3072) dtype=float32>,)
(<tf.Tensor 'MatMul_78:0' shape=(1, 128, 3072) dtype=float32>,)
(<tf.Tensor 'Add_178:0' shape=(1, 128, 3072) dtype=float32>,)
(<tf.Tensor 'Const_474:0' shape=() dtype=

(<tf.Tensor 'GatherV2_144:0' shape=(?,) dtype=int64>,)
(<tf.Tensor 'Shape_187:0' shape=(1,) dtype=int32>,)
(<tf.Tensor 'Cast_45:0' shape=(1,) dtype=int64>,)
(<tf.Tensor 'SparseToDense_45/default_value:0' shape=() dtype=int64>,)
(<tf.Tensor 'SparseToDense_45:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'add_206:0' shape=(4,) dtype=int64>,)
(<tf.Tensor 'Reshape_45:0' shape=(1, 128, 12, 64) dtype=float32>,)
(<tf.Tensor 'Shape_188:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_516:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_145/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_145:0' shape=() dtype=int64>,)
(<tf.Tensor 'Shape_189:0' shape=(3,) dtype=int64>,)
(<tf.Tensor 'Const_517:0' shape=() dtype=int64>,)
(<tf.Tensor 'GatherV2_146/axis:0' shape=() dtype=int32>,)
(<tf.Tensor 'GatherV2_146:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_518:0' shape=() dtype=int64>,)
(<tf.Tensor 'Const_519:0' shape=() dtype=int64>,)
(<tf.Tensor 'ExpandDims_183/dim:0' shape=() dtype=int32>,)
(<tf.Tensor

In [ ]:
input_tensor[0].shape